In [2]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
)

from datasets import Dataset

In [ ]:
import glob
import os
import pandas as pd
path = "14189072"  
pattern = os.path.join(path, "Russia_2_part_*.gzip.parquet")

files = glob.glob(pattern)

dfs = [pd.read_parquet(f, engine="pyarrow") for f in files]
df = pd.concat(dfs, ignore_index=True)

In [ ]:
df.to_csv("russian_disinformation_tweets.csv")

In [3]:
tweets = pd.read_csv("soci_270/russian_disinformation_tweets.csv")


print(len(tweets[tweets["is_control"] == False]))
print(len(tweets[tweets["is_control"] == True]))
tweets = tweets[["post_text", "is_control"]].dropna()
tweets = tweets.rename(columns={"post_text": "text", "is_control": "label"})

920761
90291


In [4]:
train_tweets , eval_tweets  = train_test_split(tweets , test_size=0.1, stratify=tweets["label"], random_state=42)

In [8]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
local_path = "../bert-base-uncased"

# Load the tokenizer entirely from the local directory
tokenizer = AutoTokenizer.from_pretrained(
    local_path,
    local_files_only=True
)

# Load the model entirely from the local directory
model = AutoModelForSequenceClassification.from_pretrained(
    local_path,
    local_files_only=True,
    num_labels=2
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ../bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
def tokenize_batch(batch):
    return tokenizer(batch["text"], truncation=True, max_length=256)

In [11]:
train_tweets["label"] = train_tweets["label"].astype(int)
eval_tweets["label"]  = eval_tweets["label"].astype(int)

train_ds = Dataset.from_pandas(train_tweets).map(tokenize_batch, batched=True)
eval_ds  = Dataset.from_pandas(eval_tweets).map(tokenize_batch, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map: 100%|██████████| 101106/101106 [00:25<00:00, 3918.39 examples/s]


In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    acc = accuracy_score(labels, preds)
    p, r, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")
    return {"accuracy": acc, "precision": p, "recall": r, "f1": f1}


In [ ]:
training_args = TrainingArguments(
    output_dir="./propaganda-bert-finetuned",
    eval_strategy="epoch",
    logging_strategy="steps",
    logging_steps=100,
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

metrics = trainer.evaluate(eval_ds)
print("Evaluation results:", metrics)

from sklearn.metrics import classification_report
preds_output = trainer.predict(eval_ds)
y_true = preds_output.label_ids
y_pred = np.argmax(preds_output.predictions, axis=1)
print(classification_report(y_true, y_pred, digits=4))

In [ ]:
from huggingface_hub import snapshot_download
